In [1]:
!pip install "sagemaker>=2.175.0" --upgrade --quiet

In [2]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
    
print(f'sagemaker role arn:{role}')
print(f'sagemaker session region:{sess.boto_region_name}')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn:arn:aws:iam::016767343899:role/service-role/AmazonSageMaker-ExecutionRole-20240327T155551
sagemaker session region:us-east-1


In [20]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

llm_image = get_huggingface_llm_image_uri(
    "huggingface",
    version="1.0.3"
)
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.0.3-gpu-py39-cu118-ubuntu20.04


In [21]:
import json
from sagemaker.huggingface import HuggingFaceModel

instance_type = 'ml.g5.2xlarge'
number_of_gpu = 1
health_check_timeout = 300

config = {
    'HF_MODEL_ID': "NousResearch/Llama-2-7b-chat-hf",
    'SM_NUM_GPUS': json.dumps(number_of_gpu),
    'MAX_INPUT_LENGTH': json.dumps(2048),
    'MAX_TOTAL_TOKENS': json.dumps(4096),
    'MAX_BATCH_TOYAL_TOKENS': json.dumps(8192),
    'HUGGING_FACE_HUB_TOKEN': json.dumps('HFAPI')
}

assert config['HUGGING_FACE_HUB_TOKEN'] != 'HFAPI'

llm_model = HuggingFaceModel(
    role=role,
    image_uri=llm_image,
    env=config
)

In [22]:
llm = llm_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout = health_check_timeout,
)

-----------!

In [23]:
def built_llama2_prompt(message):
    startPrompt = "<s>[INST] "
    endPrompt = " [/INST]"
    conversation = []
    for index, message in enumerate(message):
        if message["role"] == "system" and index == 0:
            conversation.append(f"<<SYS>>\n{message['content']}\n<</SYS>>\n\n")
        elif message["role"] == "user":
            conversation.append(message["content"].strip())
        else:
            conversation.append(f" [/INST] {message['content'].strip()}</s><s>[INST] ")
    
    return startPrompt + "".join(conversation) + endPrompt

message = [
    {"role":"system", "content":"you are a friendly and knowledgeable vacation planning assistant names Clara. Your goal is to have natural conversations with users."}
]
        

In [24]:
instruction = "Give me some idea what to do when I am free?"
message.append({"role":"user","content":instruction})
prompt = built_llama2_prompt(message)
chat = llm.predict({"inputs":prompt})
print(chat[0]["generated_text"][len(prompt):])

 Hello there! *smiling* Oh, wow, you must be excited to have some


In [25]:
# hyperparameters for llm
payload = {
  "inputs":  prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.9,
    "top_k": 50,
    "max_new_tokens": 512,
    "repetition_penalty": 1.03,
    "stop": ["</s>"]
  }
}

# send request to endpoint
response = llm.predict(payload)

print(response[0]["generated_text"][len(prompt):])


 Hello there! *smiling* Oh, wow, you must be excited to have some free time! *excitedly* There are so many fun things you could do, depending on your interests, of course! *nods*

Let me see... Have you ever tried visiting a local museum or art gallery? *curious* They're great places to learn something new and appreciate the culture of the area. Or, if you're feeling adventurous, you could try something like rock climbing or zip lining! *excitedly* It's a great way to get your heart racing and challenge yourself.

If you're looking for something more relaxing, you could always take a stroll through a nearby park or nature reserve. *calmly* It's a lovely way to unwind and connect with nature. Or, if you're in the mood for something more indoor-y, you could try visiting a local winery or brewery! *winks* They're always great for a fun and relaxing afternoon.

Oh, and have you thought about taking a cooking class? *excitedly* It's a great way to learn a new skill and enjoy a delicious mea